In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score, auc
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import StratifiedKFold
import category_encoders
from catboost import Pool
from catboost.utils import get_roc_curve
from sklearn.metrics import auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import catboost

In [2]:
train_data = pd.read_csv('./train_dataset/train_public.csv')
test_public = pd.read_csv('./test_public.csv')
train_internet = pd.read_csv('./train_dataset/train_internet.csv', encoding='gb2312')

In [5]:
sub=pd.read_csv("./submit/test_new_final.csv")
sub=sub.rename(columns={'id': 'loan_id'})
sub.loc[sub['isDefault']<0.4,'isDefault'] = 0
nw_sub=sub[(sub['isDefault']==0)]
nw_test_data=test_public.merge(nw_sub,on='loan_id',how='inner')
nw_train_data = pd.concat([train_data,nw_test_data]).reset_index(drop=True)
nw_train_data.to_csv("./train_dataset/nw_train_public_final.csv",index=0)

In [6]:
train_data = pd.read_csv('./train_dataset/nw_train_public_final.csv')

In [7]:
X_train = train_data.copy()
X_test = test_public.copy()
X_internet = train_internet.copy()

In [8]:
X_train = X_train.drop(['isDefault'], axis = 1)
X_train.shape

X_train['is_train'] = 1
X_test['is_train'] = 0

X_traintest = X_traintest = pd.concat([X_train, X_test], axis = 0)
X_traintest.shape

(19127, 39)

In [9]:
def time_format(x):
    #print(x)
    if x>= pd.to_datetime('2021-01-01'):
        t = '20' + str(x)[8:10] + '-' + str(x)[5:7] + '-01'
        #print('t=', t)
        return pd.to_datetime(t)
    return x

In [10]:
have_null_fea_dict = (X_traintest.isnull().sum()/len(X_traintest)).to_dict() 
fea_null_moreThanThreshold = {} 
for key,value in have_null_fea_dict.items(): 
    if value > 0.4: 
        fea_null_moreThanThreshold[key] = value
fea_null_moreThanThreshold

{}

In [11]:
unimport_fea = ['employer_type', 'industry', 'region', 'early_return', 'policy_code', 'pub_dero_bankrup', 'f1', 'del_in_18month', 'initial_list_status']
for cat in unimport_fea:
    X_internet = X_internet.drop(cat, axis=1)
    X_traintest = X_traintest.drop(cat, axis=1)

In [12]:
work_year_dict = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
}


X_traintest['work_year'] = X_traintest['work_year'].map(work_year_dict)
X_internet['work_year'] = X_internet['work_year'].map(work_year_dict)

In [13]:
class_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
}


X_traintest['class'] = X_traintest['class'].map(class_dict)
X_internet['class'] = X_internet['class'].map(class_dict)

## Date 

In [14]:
X_traintest['earlies_credit_mon'] = pd.to_datetime(X_traintest['earlies_credit_mon'])
X_internet['earlies_credit_mon'] = pd.to_datetime(X_internet['earlies_credit_mon'])

In [15]:
X_traintest['earlies_credit_mon'] = X_traintest['earlies_credit_mon'].apply(time_format)
X_internet['earlies_credit_mon'] = X_internet['earlies_credit_mon'].apply(time_format)

In [16]:
X_traintest['earlies_credit_mon_delta'] = (X_traintest['earlies_credit_mon'] - X_traintest['earlies_credit_mon'].min())  / np.timedelta64(1,'D')
X_internet['earlies_credit_mon_delta'] = (X_internet['earlies_credit_mon'] - X_internet['earlies_credit_mon'].min())  / np.timedelta64(1,'D')

In [17]:
X_traintest['issue_date'] = pd.to_datetime(X_traintest['issue_date'])
X_internet['issue_date'] = pd.to_datetime(X_internet['issue_date'])

In [18]:
X_traintest['issue_date'] = X_traintest['issue_date'].apply(time_format)
X_internet['issue_date'] = X_internet['issue_date'].apply(time_format)

In [19]:
X_traintest['issue_date_delta'] = (X_traintest['issue_date'] - X_traintest['issue_date'].min())  / np.timedelta64(1,'D')
X_internet['issue_date_delta'] = (X_internet['issue_date'] - X_internet['issue_date'].min())  / np.timedelta64(1,'D')

In [20]:
col_to_drop = ['issue_date', 'earlies_credit_mon']
X_traintest = X_traintest.drop(col_to_drop, axis=1)
X_internet = X_internet.drop(col_to_drop, axis=1)

## 交叉特征 

In [21]:
X_traintest['pro'] = X_traintest['interest'].mul(X_traintest['year_of_loan'])
X_traintest['loan_year'] = X_traintest['total_loan']/X_traintest['year_of_loan']
X_traintest['early_ratio'] = X_traintest['early_return_amount']/X_traintest['total_loan']
X_traintest['recircle_ratio'] = X_traintest['recircle_b']/X_traintest['total_loan']
X_traintest['recircle_amt'] = X_traintest['recircle_b'].mul(X_traintest['total_loan'])

In [22]:
X_internet['pro'] = X_internet['interest'].mul(X_internet['year_of_loan'])
X_internet['loan_year'] = X_internet['total_loan']/X_internet['year_of_loan']
X_internet['early_ratio'] = X_internet['early_return_amount']/X_internet['total_loan']
X_internet['recircle_ratio'] = X_internet['recircle_b']/X_internet['total_loan']
X_internet['recircle_amt'] = X_internet['recircle_b'].mul(X_internet['total_loan'])

## 特征补充 

In [23]:
numerical_fea = list(X_traintest.select_dtypes(exclude=['object']).columns) 
category_fea = list(filter(lambda x: x not in numerical_fea,list(X_traintest.columns)))

In [24]:
all_fea = X_traintest.columns
X_train = X_traintest[X_traintest.is_train == 1][all_fea]
y_train = train_data['isDefault']

X_test = X_traintest[X_traintest.is_train == 0][all_fea]

In [25]:
X_train[category_fea] = X_train[category_fea].astype(str)
X_test[category_fea] = X_test[category_fea].astype(str)

X_train[numerical_fea] = X_train[numerical_fea].astype(float)
X_test[numerical_fea] = X_test[numerical_fea].astype(float)

In [26]:
model = catboost.CatBoostClassifier(cat_features = category_fea, random_state = 2080)
model.fit(X_train, y_train, verbose=400)

Learning rate set to 0.031916
0:	learn: 0.6481321	total: 153ms	remaining: 2m 32s
400:	learn: 0.1917811	total: 2.24s	remaining: 3.34s
800:	learn: 0.1540189	total: 4.19s	remaining: 1.04s
999:	learn: 0.1386211	total: 5.16s	remaining: 0us


In [27]:
feature_importances = model.get_feature_importance(prettified=True,
                                                   thread_count=-1,
                                                   verbose=False)

In [28]:
tr_cols = set(X_traintest.columns)
same_col = list(tr_cols.intersection(set(X_internet.columns)))

tr_cols.difference(set(same_col))

{'app_type', 'is_train', 'known_dero', 'known_outstanding_loan'}

In [29]:
for i in range(feature_importances['Feature Id'].size):
    if (feature_importances['Feature Id'][i] in same_col):
        if feature_importances['Importances'][i]>1:
            X_traintest[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_traintest[feature_importances['Feature Id'][i]])
            X_internet[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_internet[feature_importances['Feature Id'][i]])


## 训练、测试数据 

In [30]:
all_fea = X_traintest.columns
X_train = X_traintest[X_traintest.is_train == 1][all_fea]
X_train = pd.concat([X_train, train_data['isDefault']], axis=1)
y_train = train_data['isDefault']

X_test = X_traintest[X_traintest.is_train == 0][all_fea]

X_train = X_train.drop('is_train', axis=1)
X_test = X_test.drop('is_train', axis=1)

In [31]:
tr_cols = set(X_train.columns)
same_col = list(tr_cols.intersection(set(X_internet.columns)))
train_inteSame = X_internet[same_col].copy()

Inte_add_cos = list(tr_cols.difference(set(same_col)))
for col in Inte_add_cos:
    train_inteSame[col] = np.nan

## Stacking 

In [32]:
clf1 = lgb.LGBMClassifier(objective='binary',
                         boosting_type='gbdt',
                         tree_learner='serial',
                         num_leaves=32,
                         max_depth=7,
                         learning_rate=0.05,
                         n_estimators=10000,
                         subsample=0.8,
                         feature_fraction=0.6,
                         reg_alpha=0.5,
                         reg_lambda=0.5,
                         random_state=2021,
                         is_unbalance=True,
                         metric='auc',
                         verbose=-1)


clf2 = catboost.CatBoostClassifier(metric_period=400,
                                   cat_features = category_fea,
                                   random_state = 2080,
                                   eval_metric='AUC')

In [33]:
def train_model(data_, test_, y_, folds_, clf):
    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in data_.columns if f not in ['loan_id', 'user_id', 'isDefault'] ]
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_)):
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]
        
        clf.fit(trn_x, trn_y, 
                eval_set= [(trn_x, trn_y), (val_x, val_y)], 
                verbose=400, early_stopping_rounds=50  #30
               )

        oof_preds[val_idx] = clf.predict_proba(val_x)[:, 1]
        sub_preds += clf.predict_proba(test_[feats])[:, 1] / folds_.n_splits
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))

        gc.collect()
        
    print('Full AUC score %.6f' % roc_auc_score(y_, oof_preds)) 
    
    test_['isDefault'] = sub_preds

    return oof_preds, test_[['loan_id', 'isDefault']], feature_importance_df

In [34]:
folds = KFold(n_splits=5, shuffle=True, random_state=546789)

In [35]:
oof_preds1, IntePre1, importances1 = train_model(X_train, train_inteSame, y_train, folds, clf1)
oof_preds2, IntePre2, importances2 = train_model(X_train, train_inteSame, y_train, folds, clf2)

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[80]	training's auc: 0.954082	valid_1's auc: 0.906008
Fold  1 AUC : 0.906008
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[24]	training's auc: 0.923355	valid_1's auc: 0.892901
Fold  2 AUC : 0.892901
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[57]	training's auc: 0.938621	valid_1's auc: 0.895941
Fold  3 AUC : 0.895941
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: featur

Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9063510136
bestIteration = 156

Shrink model to first 157 iterations.
Fold  1 AUC : 0.906351
Learning rate set to 0.05768
0:	test: 0.8559185	test1: 0.8555908	best: 0.8555908 (0)	total: 8.36ms	remaining: 8.35s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8920388499
bestIteration = 58

Shrink model to first 59 iterations.
Fold  2 AUC : 0.892039
Learning rate set to 0.057682
0:	test: 0.8537773	test1: 0.8592563	best: 0.8592563 (0)	total: 7.24ms	remaining: 7.23s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8964252234
bestIteration = 84

Shrink model to first 85 iterations.
Fold  3 AUC : 0.896425
Learning rate set to 0.057682
0:	test: 0.8608528	test1: 0.8301308	best: 0.8301308 (0)	total: 6.68ms	remaining: 6.67s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8743212214
bestIteration = 56

Shrink model to first 57 iterations.
Fold  4 AUC : 0.874321
Learning rate set to 0.057682
0:	test: 0.8579420	test1: 0.8556160	best: 0.8556160 (0)	total: 19.4ms	remaining: 19.3s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9004116825
bestIteration = 104

Shrink model to first 105 iterations.
Fold  5 AUC : 0.900412
Full AUC score 0.893727


In [36]:
IntePre1['isDef'] = train_internet['is_default']
from sklearn.metrics import roc_auc_score
roc_auc_score(IntePre1['isDef'],IntePre1.isDefault)

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.7304149890409081

In [37]:
IntePre2['isDef'] = train_internet['is_default']
from sklearn.metrics import roc_auc_score
roc_auc_score(IntePre2['isDef'],IntePre2.isDefault)

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.7365772431310529

In [38]:
IntePre = IntePre1.copy()
IntePre['isDefault'] = (IntePre['isDefault']+IntePre['isDefault'])/2

In [39]:
InteId = IntePre.loc[IntePre.isDefault<0.05, 'loan_id'].tolist()

X_train['dataSourse'] = 1
X_test['dataSourse'] = 1
train_inteSame['dataSourse'] = 0
train_inteSame['isDefault'] = X_internet['is_default']
use_te = train_inteSame[train_inteSame.loan_id.isin( InteId )].copy()
data = pd.concat([ X_train,X_test,use_te]).reset_index(drop=True)

In [40]:
train = data[data['isDefault'].notna()]
test  = data[data['isDefault'].isna()]




y = train['isDefault']
folds = KFold(n_splits=5, shuffle=True, random_state=1997)
oof_preds1, test_preds1, importances1 = train_model(train, test, y, folds, clf1)
oof_preds2, test_preds2, importances2 = train_model(train, test, y, folds, clf2)


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[55]	training's auc: 0.985193	valid_1's auc: 0.975625
Fold  1 AUC : 0.975625
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[59]	training's auc: 0.985405	valid_1's auc: 0.976566
Fold  2 AUC : 0.976566
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[114]	training's auc: 0.991802	valid_1's auc: 0.976702
Fold  3 AUC : 0.976702
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: featu

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Learning rate set to 0.082449
0:	test: 0.9567406	test1: 0.9483195	best: 0.9483195 (0)	total: 13.6ms	remaining: 13.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9773487533
bestIteration = 78

Shrink model to first 79 iterations.
Fold  1 AUC : 0.977349
Learning rate set to 0.082449


0:	test: 0.9538324	test1: 0.9588343	best: 0.9588343 (0)	total: 16.6ms	remaining: 16.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9768374268
bestIteration = 111

Shrink model to first 112 iterations.
Fold  2 AUC : 0.976837
Learning rate set to 0.08245


0:	test: 0.9540114	test1: 0.9558501	best: 0.9558501 (0)	total: 18.4ms	remaining: 18.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9774294287
bestIteration = 172

Shrink model to first 173 iterations.
Fold  3 AUC : 0.977429
Learning rate set to 0.08245


0:	test: 0.9585329	test1: 0.9424137	best: 0.9424137 (0)	total: 25.2ms	remaining: 25.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9768266097
bestIteration = 64

Shrink model to first 65 iterations.
Fold  4 AUC : 0.976827
Learning rate set to 0.08245


0:	test: 0.9539399	test1: 0.9574617	best: 0.9574617 (0)	total: 27.8ms	remaining: 27.8s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9800240482
bestIteration = 233

Shrink model to first 234 iterations.
Fold  5 AUC : 0.980024
Full AUC score 0.977619


E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
oof_preds1 = oof_preds1.reshape(-1, 1)
oof_preds2 = oof_preds2.reshape(-1, 1)


In [42]:
final_train = np.concatenate(( oof_preds1, oof_preds2), axis=1)
final_test = np.concatenate(( test_preds1[['isDefault']], test_preds2[['isDefault']]), axis=1)

In [43]:
final_test

array([[0.06012732, 0.00389082],
       [0.2753913 , 0.04177594],
       [0.01213595, 0.00152955],
       ...,
       [0.01450803, 0.00213257],
       [0.01798984, 0.01066009],
       [0.00976051, 0.00089641]])

In [44]:
final_modoel = catboost.CatBoostClassifier(metric_period=400,
                                        random_state = 1998)

In [45]:
final_modoel.fit(final_train, y)

Learning rate set to 0.059187
0:	learn: 0.4982480	total: 6.64ms	remaining: 6.63s
400:	learn: 0.0523231	total: 2.49s	remaining: 3.72s
800:	learn: 0.0500229	total: 4.79s	remaining: 1.19s
999:	learn: 0.0491874	total: 5.91s	remaining: 0us


In [46]:
predictions = final_modoel.predict_proba(final_test)[:,1]

In [47]:
predictions

array([0.00337647, 0.021529  , 0.00011629, ..., 0.00019886, 0.00399987,
       0.00011591])

In [48]:
submission = pd.read_csv('./submit_example.csv')
submission['isDefault'] = predictions
submission.to_csv('./submit/submint_final.csv', index=False)